In [ ]:
%pip install numpy pandas matplotlib scikit-learn h5py

In [1]:
import h5py
import scipy.io as sio
import scipy.stats
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Load File

In [2]:
may_12_batch = '/Users/guancong/Desktop/sch/3110_ITP/BatteryLifeTimeML/Data/2017-05-12_batchdata_updated_struct_errorcorrect.mat'
june_30_batch = '/Users/guancong/Desktop/sch/3110_ITP/BatteryLifeTimeML/Data/2017-06-30_batchdata_updated_struct_errorcorrect.mat'
f = h5py.File(may_12_batch)
f2 = h5py.File(june_30_batch)

In [16]:
batch = f['batch']
batch2 = f2['batch']

# batch 1
batch1list = []
for i in batch.keys():
    batch1list.append(batch[i])

# batch  2
batch2list = []
for i in batch2.keys():
    print(type(f2[batch2[i][1,0]]))
    batch2list.append(batch2[i])

print(batch2list)




<class 'h5py._hl.dataset.Dataset'>
<class 'h5py._hl.dataset.Dataset'>
<class 'h5py._hl.dataset.Dataset'>
<class 'h5py._hl.dataset.Dataset'>
<class 'h5py._hl.group.Group'>
<class 'h5py._hl.dataset.Dataset'>
<class 'h5py._hl.dataset.Dataset'>
<class 'h5py._hl.group.Group'>
[<HDF5 dataset "Vdlin": shape (48, 1), type "|O">, <HDF5 dataset "barcode": shape (48, 1), type "|O">, <HDF5 dataset "channel_id": shape (48, 1), type "|O">, <HDF5 dataset "cycle_life": shape (48, 1), type "|O">, <HDF5 dataset "cycles": shape (48, 1), type "|O">, <HDF5 dataset "policy": shape (48, 1), type "|O">, <HDF5 dataset "policy_readable": shape (48, 1), type "|O">, <HDF5 dataset "summary": shape (48, 1), type "|O">]


In [4]:
num_cells = batch['summary'].shape[0]
num_cells2 = batch2['summary'].shape[0]
num_cells2


48

In [23]:
# cl = batch['cycle_life'][:] # cycle life using batch group
# objectref = cl[1,0] # object reference of the first cycle life
# x = f[objectref][:] # gets the value of the cell by deferencing the object reference


# x1 = cl[0:6]
# x2 = cl[12:15]
# x3 = np.concatenate((x1,x2))

# data is represented in decimal form for strings
# print(f2[xx][:].tobytes().decode('utf-8'))

# remove batteries continuing from batch 1
batch2listupdated = []
for dset in batch2list:
    arr = dset[:]
    # skipping the batteries left out
    x1 = arr[0:7]
    x2 = arr[10:15]
    x3 = arr[17:49]
    # combined = np.concatenate((x1,x2,x3))
    combined = np.vstack((x1,x2,x3))
    batch2listupdated.append(combined)

cycles = batch2listupdated[4] # same as batch['cycle_life']
cycles = f2[cycles[0,0]] # dereferencing the object reference to get the actual data
print(type(cycles))



(43, 1)
(43, 1)
(43, 1)
(43, 1)
(43, 1)
(43, 1)
(43, 1)
(43, 1)
<class 'h5py._hl.group.Group'>


In [ ]:
#remove batteries continuing from batch 1
batch[np.array[[np.arange[8,10+1],np.arange[16,17+1]]]] = []
batch2 = batch
numBat2 = batch2.shape[2-1]
batch_combined = np.array([batch1,batch2])
numBat = numBat1 + numBat2
#remove the batteries that do not finish in Batch 1
batch_combined[np.array[[9,11,13,14,23]]] = []
numBat = numBat - 5
numBat1 = numBat1 - 5

## Outputs

In [ ]:
bat_label = np.zeros((numBat,1))
for i in np.arange(1,numBat1+1).reshape(-1):
    bat_label[i] = batch_combined(i).cycles.shape[2-1]

for i in np.arange(1,numBat2+1).reshape(-1):
    if batch2(i).summary.QDischarge(end()) < 0.88:
        bat_label[i + numBat1] = find(batch2(i).summary.QDischarge < 0.88,1)
    else:
        bat_label[i + numBat1] = batch2(i).cycles.shape[2-1]

bat_label[np.arange[1,5+1]] = np.array([[1852],[2160],[2237],[1434],[1709]])
plt.figure()
hold('on')
for i in np.arange(1,numBat+1).reshape(-1):
    plt.plot(batch_combined(i).summary.QDischarge,'.-')

max_Q = log10(np.amax(bat_label) + 1)
min_Q = log10(np.amin(bat_label) - 1)
bat_label = log10(bat_label)
colormap('jet')
CM = colormap('jet')

## Features

In [ ]:
hump = np.zeros((numBat,1))
for i in np.arange(1,numBat+1).reshape(-1):
    val,hump[i] = np.amax(batch_combined(i).summary.QDischarge(np.arange(1,100+1)))

hump[15] = 19
feat = np.zeros((numBat,1))
plt.figure()

for i in np.arange(1,numBat+1).reshape(-1):
    #Q features
    Q100_1,V100 = VQlinspace2(batch_combined(i).cycles(100))
    Q10_1,V10 = VQlinspace2(batch_combined(i).cycles(10))
    QDiff = Q100_1 - Q10_1
    feat[i,1] = log10(np.abs(var(QDiff)))

#ylim([0.95,1.1])
plt.xlabel('Q_{100} - Q_{10}')
plt.ylabel('Voltage')
set(gca,'fontsize',16)
##
colormap('jet')
CM = colormap('jet')
plt.figure()
for i in np.arange(1,feat.shape[2-1]+1).reshape(-1):
    for j in np.arange(1,numBat+1).reshape(-1):
        subplot(4,6,i)
        color_ind = np.ceil((bat_label(j) - min_Q) / (max_Q - min_Q) * 64)
        plt.plot(feat(j,i),bat_label(j),'.','Color',CM(color_ind,:),'MarkerSize',15)